# Sample Notebook

1. Setup Google Cloud service account
2. Download JSON auth keys and upload to the files
2. Share Google Drive folder with service account

In [1]:
%%capture
!pip install dvc[gdrive]
!git clone https://github.com/pavelihno/colab-dvc-template.git

In [2]:
%cd colab-dvc-template/

/content/colab-dvc-template


In [22]:
import os

GDRIVE_FOLDER_ID = '<FOLDER_ID>'
SERVICE_ACCOUNT_KEY_PATH = '/content/service-key.json'

os.environ['GDRIVE_FOLDER_ID'] = GDRIVE_FOLDER_ID
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_KEY_PATH

!dvc remote add -d gdrive "gdrive://$GDRIVE_FOLDER_ID"
!dvc remote modify gdrive gdrive_use_service_account true
!dvc remote modify gdrive gdrive_service_account_json_file_path "$GOOGLE_APPLICATION_CREDENTIALS"
!dvc pull

Setting 'gdrive' as a default remote.
Fetching
!
  0% |          |0/? [00:00<?,    ?files/s]
Fetching
Building workspace index          |7.00 [00:00,  388entry/s]
Comparing indexes          |8.00 [00:00, 1.05kentry/s]
Applying changes          |0.00 [00:00,     ?file/s]
Everything is up to date.
